# PCA

> This script compute dimensionality reduction of Simulation Dataset. Input data must be of shape **<[n_simulation, n_timesteps,3, n_nodes]**

In [ ]:
#| default_exp pca

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
import pandas as pd
import os
import yaml
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.decomposition import PCA
from argparse import ArgumentParser

In [ ]:
#| export
from dyna_PCA.get_matrix import get_matrix

In [ ]:
#| export
parser = ArgumentParser(description="Principal Components Analisys")
parser.add_argument("--file", 
                    type= str,
                    help="Path to file containing data")
parser.add_argument("--n_comp", 
                    type= float,
                    default='0.9',
                    help="Number of components [int] or amount variance in percentage (ex. 0.9 for 90% )")
parser.add_argument("--Std", 
                    type=bool,
                    default= True,
                    help="Standardize with Sklearn standard scaler")

class PrincipalComponentsAnalysis():

    def __init__(self, file,  n_comp, std):

        self.random_seed = 42
        self.file = file
        self.n_components = n_comp
        self.std = std
    
    def read_file(self):
        assert os.path.exists(self.file), "File not found"          
        self.sim = np.load(self.file)
        self.n_sim = self.sim.shape[0]
        self.timesteps = self.sim.shape[1]
        self.n_nodes = self.sim.shape[3]
        assert self.sim.shape[2] == 3, "Simulation data must be of shape [n_simulation, n_timesteps,3, n_nodes]"
        
    def train_split(self):
        self.index = np.array(range(0,self.n_sim))
        assert len(self.index) == self.n_sim 
        self.train, self.valid, self.train_idx, self.valid_idx = train_test_split(self.sim, self.index,  test_size=0.2 , shuffle=True , random_state=self.random_seed) 
            
    def get_matrix(self):
        self.train_mat = get_matrix(self.train)
        self.test_mat= get_matrix(self.valid)
    
    def p_c(self):
        scaler = StandardScaler(with_mean=True, with_std=True) 
        scaler_std = scaler.fit(self.train_mat)
        y_Data_std = scaler_std.transform(self.train_mat)

        var = PCA().fit(y_Data_std).explained_variance_
        cumulative_variance = np.cumsum(PCA().fit(y_Data_std).explained_variance_ratio_)

        pca = PCA(n_components = self.n_components)
        y_PCA = pca.fit_transform(pd.DataFrame(y_Data_std))
        n_comps = pca.n_components_

        # MSE to for Pc error
        y_PCA_inverse = pca.inverse_transform(y_PCA) # Inverse transfrom of sample data as per PCs to original form
        y_PCA_InvTrans_Data = scaler_std.inverse_transform(y_PCA_inverse, copy=None) # inverse standardization
        pca_mse = mean_squared_error(self.train_mat,y_PCA_InvTrans_Data)
        pca_mae = mean_absolute_error(self.train_mat,y_PCA_InvTrans_Data)
        print("MSE: {} for {} components:".format(pca_mse, n_comps))
        print("MAE: {} for {} components:".format(pca_mae, n_comps))
        reshape_y_pca2 =[]
        pd.DataFrame(reshape_y_pca2)
        for i in range(0,len(self.train_idx)):
            temp_1 = y_PCA[(i*(self.timesteps-1)+i):(i*(self.timesteps-1)+i+self.timesteps)].T
            temp_2 = temp_1.flatten()
            temp = pd.DataFrame(temp_2)
            reshape_y_pca2.append(temp.T)

        np.row_stack(reshape_y_pca2)   
        y_PCA_ReSh = pd.DataFrame(np.row_stack(reshape_y_pca2), index = self.train_idx) 
        y_train =y_PCA   

    def get_principal_components(self):
        self.read_file()
        self.train_split()
        self.get_matrix()
        self.p_c()

In [ ]:
#|eval: false
#|export
if __name__ == "__main__":
    args= parser.parse_args()
    print("Path to data file", args.file)
    print("Number of components [int] or amount variance in percentage [float]: ", args.n_comp)
    print("Standardization of dataset", args.std)
    obj= PrincipalComponentsAnalysis(args.file, args.n_comp, args.std)

In [ ]:
#|hide
obj= PrincipalComponentsAnalysis("../test_data/node_displacement.npy", 0.9, True)
obj.get_principal_components()

MSE: 0.008300001280836246 for n_comps: 244
MAE: 0.0726128701510692 for n_comps: 244


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()